# 리스트를 반환하는 대신 generator를 고려하자

결과를 생성하는 함수에서 택할 가장 간단한 방법은 아이템의 리스트를 반환하는 것이다.

예를 들어 문자열에 있는 모든 단어의 index를 출력하고 싶다고 할때 다음 코드에서는 append method로 리스트에 결과들을 누적하고 함수가 끝날 때 해당 리스트를 반환한다.

In [1]:
def index_words(text):
    result = []
    if text:
        result.append(0)
    for index, letter in enumerate(text):
        if letter == ' ':
            result.append(index + 1)
    return result

샘플 입력이 몇 개 뿐일 때는 함수가 기대한 대로 동작한다.

In [2]:
address = 'Four score and seven years ago...'
result = index_words(address)
print(result[:3])

[0, 5, 11]


## 위 코드의 2가지 문제

### 1. 코드가 복잡하고 깔끔하지 않다.

generator를 사용한다. generator는 yield 표현식을 사용하는 함수로 호출 될 때 실제로 실행하지 않고 바로 iterator를 반환한다. 내장 함수 next를 호출할 때마다 iterator는 generator가 다음 yield 표현식으로 진행하게 한다. __generator에서 yield에 전달한 값을 iterator가 호출하는 쪽에 반환한다.__

In [3]:
def index_words_iter(text):
    if text:
        yield 0
    for index, letter in enumerate(text):
        if letter == ' ':
            yield index + 1

result 리스트와 연동하는 부분이 모두 사라져서 훨씬 이해하기 쉽다. generator 호출로 반환되는 iterator를 내장 함수 list에 전달하면 쉽게 리스트로 변환 가능하다.

In [5]:
result = list(index_words_iter(address))

### 2. 반환하기 전에 모든 결과를 리스트에 저장해야 한다.

입력이 매우 많다면 프로그램 실행 중에 메모리가 고갈되어 동작을 멈추는 원인이 된다. __반면에 generator로 작성한 버전은 다양한 길이의 입력에도 쉽게 이용 가능하다.__

다음은 파일에서 입력을 한 번에 한 줄씩 읽어서 한 번에 한 단어씩 출력을 내어주는 generator다. 이 함수가 동작할 때 사용하는 메모리는 입력 한 줄의 최대 길이 까지다.

In [6]:
def index_file(handle):
    offset = 0
    for line in handle:
        if line:
            yield offset
        for letter in line:
            offset += 1
            if letter == ' ':
                yield offset

In [10]:
from itertools import islice

with open('/tmp/address.txt', 'w') as f:
    f.write(address)

with open('/tmp/address.txt', 'r') as f:
    it = index_file(f)
    results = islice(it, 0, 3)
    print(list(results))

[0, 5, 11]


generator를 정의할 때 알아둬야 하는 단 하나는 __반환되는 iterator에 상태가 있고 재사용할 수 없다는 사실을 호출하는 쪽에서 알아야 한다는 점__ 이다.